<a href="https://colab.research.google.com/github/Qind1609/Building_NN_from_Scrath/blob/main/Notebook_FSDL_lab0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Deep Learning Fundamentals - Part 1**

In [ ]:
!python --version

In [ ]:
!pip list | grep tensor

In [ ]:
!pip list | grep torch

In [ ]:
!nvidia-smi

***Basic numerical Computing***

In [ ]:
import numpy as np
x =np.zeros((3,2))
x

In [ ]:
x.shape, x.dtype

In [ ]:
x[0,1] = 1
x

In [ ]:
x[:,0] = 2
x

In [ ]:
x = np.array([[1,2],[3,4],[5,6]])
x

In [ ]:
X = np.array([10,20])
print(X)
print(X.shape, x.shape)

#Element-wise sum
X+x

In [ ]:
#Element-wise multiplication

X*x

In [ ]:
#matrix multiplication

X = np.array([[10, 20]]).T
mul = x @ X #or np.dot(x,X)
mul

**Indexing**

In [ ]:
y = np.random.rand(3,2) #matrix 3x2 with random element in standard distribution from 0->1
y

In [ ]:
y > 0.5 # return a mask of true and false

In [ ]:
y[y>0.5]=1 #indexing matrix
y

**Basic Ploting**


In [ ]:
import matplotlib.pyplot as plt
plt.set_cmap('gray')


In [ ]:
X = np.random.rand(100,100)
plt.matshow(X)
plt.colorbar()

In [ ]:
x = np.linspace(0,100)
y = x*5+10
plt.plot(x,y,'o-')

Basic Regression

In [ ]:
n = 50
d = 1
x = np.random.uniform(-1,1,(n,d))
weights_true = np.array([[5],])
bias_true = np.array([10])

#y = 5*x + 10
y_true = x@weights_true + bias_true
print(f'x: {x.shape}, weights: {weights_true.shape}, bias: {bias_true.shape}, y: {y_true.shape}')

plt.plot(x, y_true, marker='x',label='underlying function')
plt.legend()

**Basic the prediction: Linear**


In [ ]:
class Linear:
    #initial the prediction
    def __init__(self, num_input, num_output = 1):
        self.weights = np.random.randn(num_input, num_output)*np.sqrt(2./num_input)  #random weights
        self.bias = np.zeros((1))

    def __call__(self, x):
        return x @ self.weights + self.bias
linear = Linear(d)
y_pred = linear(x)
plt.plot(x,y_true, marker = 'x', label = 'underlying function')
plt.scatter(x,y_pred, color = 'r', marker='.', label='pre_function')
plt.legend()

**Basic loss function: MSE**

In [ ]:
#how wrong are these initial predictions?

class MSE:
    def __call__(self, y_pred,y_true):
        self.y_pred = y_pred
        self.y_true = y_true
        return ((y_true - y_pred)**2).mean()
loss = MSE()
print(f'Our initial loss is {loss(y_pred, y_true)}')

**Add back propagation**

In [ ]:
class MSE:
    def __call__(self, y_pred,y_true):
        self.y_pred = y_pred
        self.y_true = y_true
        return ((y_true - y_pred)**2).mean()
    
    def backward(self):
        n = self.y_true.shape[0]
        self.gradient = 2.*(self.y_pred - self.y_true)/n
        return self.gradient
    
class Linear:
    def __init__(self, input_dim: int, num_hidden: int = 1):
        self.weights = np.random.randn(input_dim, num_hidden) - 0.5 #distribute from -0.5 - > 0.5
        self.bias = np.random.randn(num_hidden) - 0.5

    def __call__(self, x):
        self.x = x
        output = x @ self.weights + self.bias
        return output

    #y = w*x +b
    # => dy/dx = w
    #   dy/dw = x
    # dy/db = 1

    def backward(self, gradient):
        self.weights_gradient = self.x.T @ gradient
        self.bias_gradient = gradient.sum(axis=0)
        self.x_gradient = gradient @ self.weights.T
        return self.x_gradient
    
    def update(self,lr):
        self.weights = self.weights - lr*self.weights_gradient
        self.bias = self.bias - lr*self.bias_gradient

In [ ]:
loss = MSE()
linear = Linear(d) #initalize 
y_pred = linear(x) #call
print(loss(y_pred, y_true))
loss_gradient = loss.backward()
linear.backward(loss_gradient)
linear.update(0.1)
y_pred = linear(x) #call
print(loss(y_pred, y_true))

**Training**
    

In [ ]:
plt.plot(x, y_true,marker='x', label = 'underlying function')

loss = MSE()
linear = Linear(d)

num_epochs = 60
lr = 0.1
for epoch in range(num_epochs):
    y_pred = linear(x)
    loss_value = loss(y_pred, y_true)

    if epoch % 5 == 0: #print, plot after every 5 epochs
        print(f'Epoch {epoch}, loss {loss_value}')
        plt.plot(x, y_pred.squeeze(), label = f'Epoch {epoch}')
    gradient_from_loss = loss.backward()
    linear.backward(gradient_from_loss)
    linear.update(lr)

plt.legend(bbox_to_anchor = (1.04,1), loc ="upper left")

In [ ]:
# 2-D of x

n = 100
d = 2
x = np.random.uniform(-1,1,(n,d))
# y = w*x + b
# y = w0 * x0 + w1 * x1 + b
# y = w@x +b

weights_true = np.array([[2,-1]]).T  
bias_true = np.array([0.5])  #y = 2*x0 - x1 + 0.5
print(x.shape, weights_true.shape, bias_true.shape)

y_true =  x @ weights_true + bias_true
print(f'x: {x.shape}, weights: {weights_true.shape}, bias: {bias_true.shape}, y: {y_true.shape}')

def plot_3d(x,y,y_pred = None):
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure()
    ax = fig.add_subplot(111,projection = '3d')
    ax.scatter(x[:,0],x[:,1],y, label = 'underlying function')
    if y_pred is not None:
        ax.scatter(x[:,0],x[:,1], y_pred, label = 'our function')
    plt.legend()

plot_3d(x,y_true)

In [ ]:
loss = MSE()
linear = Linear(2)
y_pred = linear(x)
print(loss(y_pred, y_true))
fig = plot_3d(x,y_true,y_pred)

In [ ]:
from typing import Callable
def fit(x: np.ndarray, y: np.ndarray, model: Callable, loss: Callable, lr: float, num_epochs: int):
    for epoch in range(num_epochs):
        y_pred = model(x)
        loss_value = loss(y_pred, y)
        print(f'Epoch {epoch}, loss {loss_value}')
        gradient_from_loss = loss.backward()
        model.backward(gradient_from_loss)
        model.update(lr)

fit(x,y_true,model = linear, loss=loss, lr = 0.1, num_epochs = 60)
plot_3d(x,y_true, linear(x))

**Basic Regression with a Multi-layer Perceptron or Neural Network**

In [ ]:
#Non linear function
n = 200
d = 2 
x = np.random.uniform(-1,1,(n,d))

weights_true = np.array([[5,1]]).T
bias_true = np.array([10])

y_true = (x**2) @ weights_true + x @ weights_true + bias_true
print(f'x: {x.shape}, weights: {weights_true.shape}, bias: {bias_true.shape}, y: {y_true.shape}')

plot_3d(x,y_true)

In [ ]:
#try to approximate this function by linear regression

loss = MSE()
linear = Linear(d)
fit(x, y_true, model = linear, loss = loss, lr = 0.1, num_epochs = 40)
plot_3d(x,y_true, linear(x))

**Using Non-Linearity : ReLu**

In [ ]:
class ReLu:
    def __call__(self, input_):
        self.input_ = input_
        self.output = np.clip(self.input_,0,None)
        return self.output
        
    def backward(self, output_gradient):
        self.input_gradient = (self.input_ > 0) * output_gradient
        return self.input_gradient

relu = ReLu()
input_ = np.expand_dims(np.array([1,0.5,0,-0.5,-1]),-1)
print(relu(input_))
print(relu.backward(input_))

In [ ]:
class Model:
    def __init__(self, input_dim, num_hidden):
        self.linear1 = Linear(input_dim, num_hidden)  #multiply matrix random weights 
        self.relu = ReLu()
        self.linear2 = Linear(num_hidden, 1)       # mtrix random weights
    
    def __call__(self, x):  
        l1 = self.linear1(x)
        r = self.relu(l1)
        l2 = self.linear2(r)
        return l2

    def backward(self, output_gradient):
        linear2_gradient = self.linear2.backward(output_gradient)
        relu_gradient = self.relu.backward(linear2_gradient)
        linear1_gradient = self.linear1.backward(relu_gradient)
        return linear1_gradient
    
    def update(self, lr):
        self.linear2.update(lr)
        self.linear1.update(lr)

#test just one forward and backward step

loss = MSE()
model = Model(d,10)
y_pred = model(x)
loss_value = loss(y_pred,y_true)
loss_gradient = loss.backward()
print(loss_value)
model.backward(loss_gradient)
plot_3d(x,y_true,y_pred)

In [ ]:
#training
fit(x,y_true, model=model,loss = loss , lr = 0.1, num_epochs = 200)
plot_3d(x,y_true, model(x))


**Try with Pytorch instead of coding from scrath**


In [ ]:
import torch
import torch.nn as nn

class TorchModel(nn.Module):
    def __init__(self, input_dim, num_hidden):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, num_hidden)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(num_hidden,1)

    def forward(self, x):
        l1 = self.linear1(x)
        r  = self.relu(l1)
        l2 = self.linear2(r)
        return l2

#inital loss
loss = nn.MSELoss()
model = TorchModel(d,10)
x_tensor = torch.tensor(x).float()
y_true_tensor = torch.tensor(y_true).float()
y_pred_tensor = model(x_tensor) #call forward
loss_value = loss(y_pred_tensor, y_true_tensor)
print(loss_value)
plot_3d(x_tensor, y_true_tensor, y_pred_tensor.detach())

In [ ]:
#Test just one forward and backward step

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

optimizer.zero_grad()
y_pred_tensor = model(x_tensor)
loss_value = loss(y_pred_tensor, y_true_tensor)
print(loss_value)
loss_gradient = loss_value.backward()
optimizer.step()

y_pred_tensor = model(x_tensor)
loss_value = loss(y_pred_tensor, y_true_tensor)
print(loss_value)

In [ ]:
def torch_fit(x: np.ndarray, y: np.ndarray, model: Callable, loss: Callable, lr: float, num_epochs: int):
    optimizer = torch.optim.SGD(model.parameters(), lr = lr)
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        y_pred_tensor = model(x_tensor)
        loss_value = loss(y_pred_tensor, y_true_tensor)
        print(loss_value)
        loss_value.backward()
        optimizer.step()
torch_fit(x_tensor, y_true_tensor, model = model, loss=loss, lr = 0.1, num_epochs = 40)
plot_3d(x, y_true, linear(x))

 

**Try with TensorFlow/Keras**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers

inputs = keras.Input(shape=(2,))
l1 = layers.Dense(10, activation = 'relu', name = 'dense_1')(inputs)
outputs = layers.Dense(1, name='regression')(l1)

model = keras.Model(inputs = inputs, outputs = outputs)

print(model.summary())
model.compile(loss = 'mse', optimizer = optimizers.SGD(0.1))

model.fit(x,y_true, epochs = 60)

y_pred = model.predict(x)
plot_3d(x,y_true, model(x))